In [1]:
import cv2 as cv
import os
import tensorrt as trt
import logging as logger


In [2]:
trt.__version__

'10.1.0'

In [3]:
def load_trt_engine(engine_path):
        '''
        Load TRT engine
        '''
        TRT_LOGGER = trt.Logger(trt.Logger.INFO)
        if not os.path.exists(engine_path):
            logger.error(f"Engine file not found at {engine_path}")
            return None
        
        with open(engine_path, 'rb') as f, trt.Runtime(TRT_LOGGER) as runtime:
            try:
                engine = runtime.deserialize_cuda_engine(f.read())
                if engine is None:
                    script_dir = os.path.dirname(os.path.realpath(__file__))
                    logger.error(f"Failed to load engine from {script_dir}")
                return engine
            except Exception as e:
                logger.error(f"Exception while loading engine: {e}")
                return None
    
def get_intrinsics(file_path):
    # Open the file storage for reading
    cv_file = cv.FileStorage(file_path, cv.FILE_STORAGE_READ)

    intrinsics = {}
    # Read the left stereo map (x coordinates)
    intrinsics['Left_Stereo_Map_x'] = cv_file.getNode("Left_Stereo_Map_x").mat()

    # Read the left stereo map (y coordinates)
    intrinsics['Left_Stereo_Map_y'] = cv_file.getNode("Left_Stereo_Map_y").mat()

    # Read the right stereo map (x coordinates)
    intrinsics['Right_Stereo_Map_x'] = cv_file.getNode("Right_Stereo_Map_x").mat()

    # Read the right stereo map (y coordinates)
    intrinsics['Right_Stereo_Map_y'] = cv_file.getNode("Right_Stereo_Map_y").mat()

    # Read the rectified camera matrix
    intrinsics['Rectifyed_mat_left'] = cv_file.getNode("Rectifyed_mat_left").mat()
    intrinsics['Mat_left'] = cv_file.getNode("Mat_left").mat()

    # Read the baseline (distance between the two cameras)
    intrinsics['Baseline'] = cv_file.getNode("Baseline").real()

    return intrinsics

In [4]:
engine = load_trt_engine('/var/model_converter/model.engine')
if engine is None:
    logger.error("Failed to load TensorRT engine.")


execution_context = engine.create_execution_context()
if execution_context is None:
    logger.error("Failed to create TensorRT execution context.")
intrinsics = get_intrinsics('/var/camera_params/intrinsics.xml')

[08/14/2024-14:28:12] [TRT] [I] Loaded engine size: 7 MiB
[08/14/2024-14:28:12] [TRT] [W] Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.
[08/14/2024-14:28:12] [TRT] [I] [MS] Running engine with multi stream info
[08/14/2024-14:28:12] [TRT] [I] [MS] Number of aux streams is 2
[08/14/2024-14:28:12] [TRT] [I] [MS] Number of total worker streams is 3
[08/14/2024-14:28:12] [TRT] [I] [MS] The main stream provided by execute/enqueue calls is the first worker stream
[08/14/2024-14:28:12] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +690, now: CPU 0, GPU 764 (MiB)


In [5]:
# Inspect the inputs of the TensorRT engine
def inspect_inputs(engine):
    for binding in engine:
        if engine.binding_is_input(binding):
            input_shape = engine.get_binding_shape(binding)
            input_dtype = engine.get_binding_dtype(binding)
            print(f"Input Name: {binding}")
            print(f"Input Shape: {input_shape}")
            print(f"Input Data Type: {input_dtype}\n")

In [26]:
engine.get_tensor_shape(engine.get_tensor_name(0))

720